# Import Dependencies

In [1]:
import numpy as np
import pandas as pd
from math import *
from scipy.integrate import quad
from scipy.optimize import root

# Data Utilizations

In [2]:
file1 = 'ft_fn_fb.xlsx'
file2 = 'K.xlsx'

In [3]:
def write_excel(df, file_name):
    file_path = pd.ExcelWriter(file_name)
    df.fillna(' ', inplace=True)
    df.to_excel(file_path, encoding='utf-8', index=False)
    file_path.save()
    print("The excel is successfully written!")

# Fourth-order Runge-Kutta Method
## Set Controlled Variables

In [4]:
Theta = [radians(angle) for angle in range(10,91,10)]
T = [t for t in range(283,474,10)]
print(f'Theta :{Theta[:3]}\nT     :{T[:3]}')

Theta :[0.17453292519943295, 0.3490658503988659, 0.5235987755982988]
T     :[283, 293, 303]


## Calculate Constants

In [5]:
EPSILON = 0.000001

def calculate_R(theta):
    R = 1
    return R

def calculate_kappa(theta, R):
     return sin(theta)**2/R

def calculate_tau(theta, R):
     return sin(theta)*cos(theta)/R

## Define RK Method

In [6]:
def RK4(F, s, y, h):
    k1 = h * np.array(F(s, y))
    k2 = h * np.array(F(s + h/2, y + k1/2))
    k3 = h * np.array(F(s + h/2, y + k2/2))
    k4 = h * np.array(F(s + h, y + k3))
    return y + (k1 + 2*k2 + 2*k3 + k4)/6

## Define & Solve ODE

### Definition

In [7]:
# Define Solve
def solve(F, F_T0, F_N0, F_B0, s_start, s_end, h):
    s = s_start
    F_T = F_T0
    F_N = F_N0
    F_B = F_B0
    while s < s_end:
        y = [F_T, F_N, F_B]
        y = RK4(F, s, y, h)
        F_T, F_N, F_B = y
        s += h
    return F_T, F_N, F_B

### Solve

In [8]:
F_T0 = 0
F_N0 = 0
F_B0 = 0
s_start = 0
s_end =  1
h = 0.01

res_list = []
for t in T:
    for theta in Theta:
        
        # Define ODE
        def F(s, y): 
            F_T, F_N, F_B = y
            global t
            global theta
            t = t #t and theta are the two controlled vars.
            theta = theta
            R = calculate_R(theta)
            tau = 3e7 #Tau max is set as an constant value of 30MPa.
            r = 9e-11
            a = 7.5e-4
            miu = 0.28
            E = 9.7e11
            kappa = calculate_kappa(theta, R)
            tau = calculate_tau(theta, R)
            f_T = 2*pi*r*tau
            f_N = 2*pi*E*r*a*t/(2-miu)
            f_B = -2*pi*E*r*a*t/(2-miu)
            dF_Tds = F_N * kappa - f_T
            dF_Nds = F_B * tau - F_T * kappa - f_N
            dF_Bds = -F_N * tau - f_B
            return [dF_Tds, dF_Nds, dF_Bds]
        
        tmp_dict = {'T':t, 'theta':theta}
        res = solve(F, F_T0, F_N0, F_B0, s_start, s_end, h)
        tmp_dict['F_T'] = res[0]
        tmp_dict['F_N'] = res[1]
        tmp_dict['F_B'] = res[2]
        res_list.append(tmp_dict)

In [9]:
rk_df = pd.DataFrame(res_list)
rk_df.shape
rk_df.head()

,T,theta,F_T,F_N,F_B
0,283,0.174533,-0.959878,-61.575441,73.132000
1,283,0.349066,-3.498908,-55.604616,77.301434
2,283,0.523599,-7.080154,-50.550803,79.951448
3,283,0.698132,-11.260451,-47.023393,81.107946
4,283,0.872665,-15.742648,-45.393825,80.897912


In [10]:
# Get the results of ft, fn and fb.
write_excel(rk_df, file1)

The excel is successfully written!


## Test RK

### Single Equation 

In [11]:
# Define ODE
def F_test(s, y):
    return -y+1

# Define Solve
def solve_test(F, y0, s_start, s_end, h):
    s = s_start
    y = y0
    while s < s_end:
        y = y
        y = RK4(F, s, y, h)
        y = y
        s += h
    return y

In [12]:
# Solve
y0 = 0.0
s0 = 0.0

s1 = 0.1
h = 0.001
y1 = solve_test(F_test,y0,s0,s1,h)
print("y(0.1) = ", y1)

y(0.1) =  0.09516258196403962


### Equation Set

In [13]:
# Define ODE
def F_test_2(s, y):
    u,v = y
    duds = u + 2*v
    dvds = 3*u + 2*v
    return [duds, dvds]
    
# Define Solve
def solve_test_2(F, u0, v0, s_start, s_end, h):
    s = s_start
    u = u0
    v = v0
    while s < s_end:
        y = [u, v]
        y = RK4(F, s, y, h)
        u, v = y
        s += h
    return u, v

In [14]:
# Solve
u0 = 6
v0 = 4
s0 = 0

s1 = 0.2
h = 0.01
u, v = solve_test_2(F_test_2, u0, v0, s0, s1, h)
print("u(0.2) = ", u)
print("v(0.2) = ", v)

u(0.2) =  10.539625073201622
v(0.2) =  11.715783844343727


# Calculate COD

## Set Constants

In [15]:
x0=10e-9
n=10e-9 #Δx
m=1860e-9 # lambda
x = np.arange(x0 - n/2, m - n/2, n)
s = len(x)
E=9.7e11
K=1e6
tau=30e6
V=0.4
R=14e-9 # rc
p=1e-11
l=1.5e-6 # Undecided yet!

## Calculate δ(xi)

In [16]:
res_list = []
for theta in Theta:
    res_dict = {'theta':theta}
    
    # Set up coefs.
    a=4*V*tau*n*cos(theta)/(pi*E*R) # Relevant to theta.
    b=8*K*sqrt(1-V)/(E*sqrt(2*pi))
    c=4*V*l*tau*cos(theta)/(pi*E*R) # Relevant to theta.
    
    # Set up matrixs.
    Y = np.ndarray(shape=(s, s))
    z = np.ndarray(shape=(s,))
    U = np.ndarray(shape=(s,))
    
    # Consider the calculation of δ(xi) as a problem of Yx=z.
    # Calculate Y.
    for i in range(s):
        for j in range(s):
            if j != i:
                Y[i,j] = a * (2 * np.sqrt(m - x[i]) / np.sqrt(m - x[j]) - np.log((np.sqrt(m - x[i]) + np.sqrt(m - x[j])) / np.abs(np.sqrt(m - x[i]) - np.sqrt(m - x[j]))))
            else:
                Y[j,j] = 1 - a * (np.log((np.sqrt(m - x[j]) + np.sqrt(m - x[j] - p)) / np.abs(np.sqrt(m - x[j]) - np.sqrt(m - x[j] - p))) - 2)
    
    # Calculate z.
    for i in range(s):
        z[i] = b * np.sqrt(m - x[i]) + c * (7 * np.sqrt(m * (m - x[i])) - (m+x[i]) * np.log((np.sqrt(m) + np.sqrt(m - x[i])) / (np.sqrt(m) - np.sqrt(m - x[i]))))
    
    # Use inverse matrix to solve the linear equations.
    z = z.T
    D = np.linalg.inv(Y)@z
    
    # Calculate Kb
    W = D.T
    F = 0
    for i in range(s):
        U[i] = W[i]*n/sqrt(m-x[i])
        F+=U[i]
    Kb=sqrt(2/pi)*V*cos(theta)*tau*(4*sqrt(m)*l-F)/R
    res_dict['Kb'] = Kb
    res_list.append(res_dict)

In [17]:
lu_df = pd.DataFrame(res_list)
lu_df.shape
lu_df.head()

,theta,Kb
0,0.174533,5.499491e+06
1,0.349066,5.247930e+06
2,0.523599,4.837085e+06
3,0.698132,4.279334e+06
4,0.872665,3.591491e+06


In [18]:
# Get the results of Kb.
write_excel(lu_df, file2)

The excel is successfully written!
